Python version: 3.10.13

In [2]:
import pandas as pd



df1 = pd.read_json("/Users/gm/Desktop/Spotify/MyData 2/Streaming_History_Audio_2016-2018_0.json") # Load the JSON File into a dataframe
df2 = pd.read_json("/Users/gm/Desktop/Spotify/MyData 2/Streaming_History_Audio_2018-2019_1.json")
df3 = pd.read_json("/Users/gm/Desktop/Spotify/MyData 2/Streaming_History_Audio_2019-2021_2.json")
df4 = pd.read_json("/Users/gm/Desktop/Spotify/MyData 2/Streaming_History_Audio_2021-2022_3.json")
df5 = pd.read_json("/Users/gm/Desktop/Spotify/MyData 2/Streaming_History_Audio_2022-2023_4.json")
df6 = pd.read_json("/Users/gm/Desktop/Spotify/MyData 2/Streaming_History_Audio_2023_5.json")

spotify_stream_df = pd.concat([df1,df2,df3,df4,df5,df6], ignore_index=True) # To concatenate the two dataframes